In [ ]:
import pandas as pd

# Load data
df = pd.read_csv("../input/liver_qc.cvs")

# Apply CAP filter
cap = 248
df = df[df["CAP Enhanced Mean (dB/m)"] > cap].copy()
#1 ID was mismatched between manual and the program for 248 and 263 cap score
#this id was not included in the manual bcs the we did not included the hdl X chol med combination as it is not biologically valid
# we should only check the chol med with triglycerides
# Define individual binary condition columns
df["BMI_cond"] = (df['Final_bmi'] >= 23).astype(int)
df["Male_waist_90"] = ((df['Final_gender'] == 'Male') & (df['average_waist'] >= 90)).astype(int)
df["Male_hdl_40"] = ((df['Final_gender'] == 'Male') & (df['Serum HDL Cholesterol'] < 40)).astype(int)
df["Female_waist_80"] = ((df['Final_gender'] == 'Female') & (df['average_waist'] >= 80)).astype(int)
df["Female_hdl_50"] = ((df['Final_gender'] == 'Female') & (df['Serum HDL Cholesterol'] < 50)).astype(int)
df["Triglycerides_cond"] = ((df['Serum Triglycerides'] >= 150) | (df['medHis_highChol_medicine'] == 'Yes') ).astype(int)
df["Diabetes_cond"] = (df['Diabetic_final'] == 'Yes').astype(int)
df["Prediabetes_cond"] = (df['Prediabetics_final'] == 'Yes').astype(int)
df["Hypertension_cond"] = (df['Hypertension_final'] == 'Yes').astype(int)
#df["Chol_med_cond"] = (df['medHis_highChol_medicine'] == 'Yes').astype(int)
df["HOMA_IR_2.5"] = (df["HOMA IR"]>=2.5).astype(int)
# List of secondary conditions for summing
secondary_condition_cols = [
    "Male_waist_90", "Male_hdl_40", "Female_waist_80",
    "Female_hdl_50", "Triglycerides_cond", "Prediabetes_cond",
    "Hypertension_cond", "HOMA_IR_2.5"
]

# Sum of secondary conditions per row
df["condition_sum"] = df[secondary_condition_cols].sum(axis=1)

# Apply final MAFLD criteria:
# MASLD = 1 if (Diabetes == Yes) OR (BMI ≥ 23) OR (sum of secondary conditions ≥ 2)
df[f"MAFLD_{cap}"] = (
    (df["Diabetes_cond"] == 1) |
    (df["BMI_cond"] == 1) |
    (df["condition_sum"] >= 2)
).astype(int)

# Save output
df.to_csv(f"../output/MAFLD_matrix_{cap}.csv", index=False)

# Summary
print("Rows with MAFLD:", df[f"MAFLD_{cap}"].sum())

/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_57752/1145321760.py:4: DtypeWarning: Columns (4,6,10,11,12,13,15,17,19,20,21,22,23,24,25,31,37,39,43,49,109,110,112,113,114,115,116,117,120,121,122,123,188,193,203,213,221,224,228,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../input/liver_qc.cvs")


Rows with MAFLD: 4802


In [17]:
# Load main DataFrame
df = pd.read_csv("../input/liver_qc.cvs")

# Load and trim other DataFrames
df2 = pd.read_csv("../output/MAFLD_matrix_248.csv")[["LABID", "MAFLD_248"]]
df3 = pd.read_csv("../output/MAFLD_matrix_263.csv")[["LABID", "MAFLD_263"]]
df4 = pd.read_csv("../output/MAFLD_matrix_275.csv")[["LABID", "MAFLD_275"]]

# Merge all on LABID (left join)
df = df.merge(df2, on="LABID", how="left")
df = df.merge(df3, on="LABID", how="left")
df = df.merge(df4, on="LABID", how="left")

# Optional: Save merged result
df.to_csv("../output/liver_with_MAFLD_flags.csv", index=False)

/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_57752/1976460562.py:2: DtypeWarning: Columns (4,6,10,11,12,13,15,17,19,20,21,22,23,24,25,31,37,39,43,49,109,110,112,113,114,115,116,117,120,121,122,123,188,193,203,213,221,224,228,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../input/liver_qc.cvs")
/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_57752/1976460562.py:5: DtypeWarning: Columns (6,10,11,12,13,15,16,39,43,109,110,112,113,115,117,118,121,122,123,187,188,193,200,215,216,221,222,223,224,226,228,229,231,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../output/MAFLD_matrix_248.csv")[["LABID", "MAFLD_248"]]
/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_57752/1976460562.py:6: DtypeWarning: Columns (6,10,11,12,13,15,16,39,43,112,113,121,122,224) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("../outp